In [ ]:
import csv
import pandas as pd
import itertools
import math
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
import string
import re
import nltk
import random
from google.colab import files
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import classification_report, plot_confusion_matrix

In [ ]:
pip install praw

     |████████████████████████████████| 153kB 2.7MB/s 
     |████████████████████████████████| 204kB 9.0MB/s 


In [ ]:
import praw

In [ ]:
city_state_csv = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs27-C-HRF-DS/main/project/cities_states.csv')

city_state_df = pd.DataFrame(city_state_csv)

city_state_df.head()

,Unnamed: 0,city_ascii,admin_name,country,lat,lng
0,1,New York,New York,United States,40.6943,-73.9249
1,8,Los Angeles,California,United States,34.1139,-118.4068
2,24,Chicago,Illinois,United States,41.8373,-87.6862
3,40,Miami,Florida,United States,25.7839,-80.2102
4,42,Dallas,Texas,United States,32.7936,-96.7662


In [ ]:
# DON'T USE THIS CELL UNTIL YOU KNOW WHAT YOU'RE DOING!!

def run_update() -> None:
    '''
    Update backlog database with data from reddit.
    '''
    print('Updating backlog at %s' % datetime.now())
    # reddit = praw.Reddit(client_id='97nXmEBrjK3CIw', client_secret="WIE6O_ECbv6zaSR3m8au8fWrnwc", password="Gwill123", user_agent='hrf-labs27', username='griffinwilson')
    PRAW_CLIENT_ID = '97nXmEBrjK3CIw'
    PRAW_CLIENT_SECRET = 'WIE6O_ECbv6zaSR3m8au8fWrnwc'
    PRAW_USER_AGENT = 'hrf-labs27'

    reddit = praw.Reddit(
        client_id=PRAW_CLIENT_ID,
        client_secret=PRAW_CLIENT_SECRET,
        user_agent=PRAW_USER_AGENT
    )
    # Grab data from reddit
    data = []
    for submission in reddit.subreddit("news").top('week', limit=500):
        data.append([
            submission.id, submission.title, submission.url
        ])
    # construct a dataframe with the data
    col_names = ['id', 'title', 'url']
    df = pd.DataFrame(data, columns=col_names)

    # pull the text from each article itself using newspaper3k
    content_list = []
    date_list = []
    # go through each URL and use newspaper3k to extract data
    for id_url in df['url']:
        # use newspaper3k to extract text
        article = Article(id_url)
        article.download()
        # if the article doesn't download, the error is thrown in parse()
        try:
            article.parse()
        except:
            # add null values to show no connection
            content_list.append(None)
            date_list.append(None)
            continue
        content_list.append(article.text)
        # this will be null if newspaper3k can't find it
        date_list.append(article.publish_date)
    df['text'] = content_list
    df['date'] = date_list

    # drop any articles with missing data columns
    df = df.dropna()
    df = df.reset_index()
    df = df.drop(columns='index')

    # convert date column to pandas Timestamps
    def timestampify(date):
        return pd.Timestamp(date, unit='s').isoformat()
    df['date'] = df['date'].apply(timestampify)

    # use NLP model to filter posts
    df['is_police_brutality'] = pipeline.predict(df['title'])
    df = df[df['is_police_brutality'] == 1]
    df = df.drop(columns='is_police_brutality')

    # use spaCy to extract location tokens
    tokens_list = []
    for text in df['text']:
        doc = nlp(text.lower())
        ents = [e.text for e in doc.ents if e.label_ == 'GPE']
        tokens_list.append(ents)
    df['tokens'] = tokens_list

    # figure out which city and state the article takes place in
    city_list = []
    state_list = []
    geo_list = []
    for tokens in df['tokens']:
        # set up Counter
        c = Counter(tokens)

        # set up geolocation dict for geo list
        geo_entry = {'lat': None, 'long': None}

        # count which states come back the most, if any
        state_counts = {}
        for state in states_map:
            if c[state] > 0:
                state_counts[state] = c[state]

        # get state(s) that came back the most as dict with lists
        max_count = 0
        max_state = None

        for state in state_counts:
            if state_counts[state] > max_count:
                max_count = state_counts[state]
                max_state = {state: {}}
            elif state_counts[state] == max_count:
                max_state[state] = {}

        # if no state is found
        if max_state is None:
            city_list.append(None)
            state_list.append(None)
            geo_list.append(geo_entry)
            continue

        max_city = None
        # get any cities in tokens based on states
        for state in max_state:  # ideally this should only run once
            city_counts = {}
            for city in states_map[state]:
                if c[city] > 0:
                    city_counts[city] = c[city]
            max_state[state] = city_counts

            # get the city/state combo that came back the most
            max_count = 0
            for city in city_counts:
                if city_counts[city] > max_count:
                    max_count = city_counts[city]
                    max_city = (city, state)

        # if no city is found
        if max_city is None:
            city_list.append(None)
            state_list.append(None)
            geo_list.append(geo_entry)
            continue

        # the city and state should be known now

        city_list.append(max_city[0].title())
        state_list.append(max_city[1].title())
        # now get the geolocation data
        row = locs_df[(
            (locs_df['city_ascii'] == max_city[0]) &
            (locs_df['admin_name'] == max_city[1])
        )]
        row = row.reset_index()
        if row.empty:
            pass
        else:
            geo_entry['lat'] = row['lat'][0]
            geo_entry['long'] = row['lng'][0]
        # geo_list.append(geo_entry)

    # loop ends, add cities and states onto dataframe
    df['city'] = city_list
    df['state'] = state_list
    df['lat'] = geo_entry['lat']
    df['long'] = geo_entry['long']



In [ ]:
# Create new news.csv

!pip install praw

In [ ]:
####  REDDIT CREDENTIALS HERE!

reddit = praw.Reddit(client_id='97nXmEBrjK3CIw', client_secret="WIE6O_ECbv6zaSR3m8au8fWrnwc", password="Gwill123", user_agent='hrf-labs27', username='griffinwilson')

In [ ]:
# Grabbing 1000 hottest posts on Reddit 

data = []

# Grab the data from the "news" subreddit
for submission in reddit.subreddit("news").hot(limit=1000):
  data.append([submission.id, submission.title, submission.score, submission.subreddit, submission.url, 
               submission.num_comments, submission.selftext, submission.created])

# Create and assign column names
col_names = ['id', 'title', 'score', 'subreddit', 'src', 
             'num_comments', 'desc', 'created']
df_reddit = pd.DataFrame(data, columns=col_names)

df_reddit.head()

,id,title,score,subreddit,src,num_comments,desc,created
0,j5m8qf,President Trump and US Government COVID-19 Meg...,2141,news,https://www.reddit.com/r/news/comments/j5m8qf/...,14679,**This thread is for discussing all things rel...,1.601943e+09
1,jadmis,Dr. Anthony Fauci says he won't walk away from...,56649,news,https://www.cnbc.com/2020/10/13/dr-anthony-fau...,1817,,1.602625e+09
2,jadfuu,"Breonna Taylor's boyfriend, Kenneth Walker: ""I...",63063,news,https://www.cbsnews.com/news/breonna-taylor-bo...,896,,1.602625e+09
3,jagzjo,Facebook says it will finally ban anti-vaccina...,4193,news,http://www.cnbc.com/2020/10/13/facebook-bans-a...,258,,1.602636e+09
4,jahsmb,"Judge rules California inmates can receive $1,...",2870,news,https://www.whio.com/news/trending/judge-rules...,309,,1.602638e+09


In [ ]:
import datetime

def get_date(time):
	return datetime.date.fromtimestamp(time)
 
df_reddit['date'] = df_reddit['created'].apply(get_date)
df_reddit['lat'] = ""
df_reddit['long'] = ""

df_reddit.head()

,id,title,score,subreddit,src,num_comments,desc,created,date,lat,long
0,j5m8qf,President Trump and US Government COVID-19 Meg...,2141,news,https://www.reddit.com/r/news/comments/j5m8qf/...,14679,**This thread is for discussing all things rel...,1.601943e+09,2020-10-06,,
1,jadmis,Dr. Anthony Fauci says he won't walk away from...,56649,news,https://www.cnbc.com/2020/10/13/dr-anthony-fau...,1817,,1.602625e+09,2020-10-13,,
2,jadfuu,"Breonna Taylor's boyfriend, Kenneth Walker: ""I...",63063,news,https://www.cbsnews.com/news/breonna-taylor-bo...,896,,1.602625e+09,2020-10-13,,
3,jagzjo,Facebook says it will finally ban anti-vaccina...,4193,news,http://www.cnbc.com/2020/10/13/facebook-bans-a...,258,,1.602636e+09,2020-10-14,,
4,jahsmb,"Judge rules California inmates can receive $1,...",2870,news,https://www.whio.com/news/trending/judge-rules...,309,,1.602638e+09,2020-10-14,,


In [ ]:
# Create dataframe using 2020PB source

# Import and normalize the json file 
df_2020pb = pd.read_json('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations-v2.json')
df_2020pb = pd.json_normalize(df_2020pb['data'])
df_2020pb = df_2020pb.drop(columns=['edit_at'])
df_2020pb = df_2020pb.rename(columns={'description':'desc', 'links':'src', 'name':'title'})
df_2020pb['lat'] = ""
df_2020pb['long'] = ""

df_2020pb.head()

,src,state,city,desc,tags,geolocation,title,date,date_text,id,lat,long
0,[{'url': 'https://www.youtube.com/watch?v=s7MM...,Washington,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st,wa-olympia-1,,
1,[{'url': 'https://mobile.twitter.com/chadloder...,Washington,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th,wa-seattle-1,,
2,[{'url': 'https://twitter.com/gunduzbaba1905/s...,Washington,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2,,
3,[{'url': 'https://www.reddit.com/r/Bad_Cop_No_...,Washington,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st,wa-seattle-3,,
4,[{'url': 'https://www.fox10phoenix.com/news/vi...,Washington,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4,,


In [ ]:
df_2020pb['src'][1]

[{'text': '',
  'url': 'https://mobile.twitter.com/chadloder/status/1267011092045115392'}]

In [ ]:
import re

def remove_url(df):
  for i in range(len(df)):
    # initializing split word 
    df['src'][i] = str(df['src'][i])

    spl_word = "'url': "

    res = df['src'][i].partition(spl_word)[2] 
    res = res.replace('\'', '')

    df['src'][i] = res.split(',')[0]

In [ ]:
remove_url(df_2020pb)

df_2020pb.head()

,src,state,city,desc,tags,geolocation,title,date,date_text,id,lat,long
0,https://www.youtube.com/watch?v=s7MM1VauRHo,Washington,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st,wa-olympia-1,,
1,https://mobile.twitter.com/chadloder/status/12...,Washington,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th,wa-seattle-1,,
2,https://twitter.com/gunduzbaba1905/status/1266...,Washington,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2,,
3,https://www.reddit.com/r/Bad_Cop_No_Donut/comm...,Washington,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st,wa-seattle-3,,
4,https://www.fox10phoenix.com/news/video-shows-...,Washington,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4,,


In [ ]:
df_2020pb['src'][1]

'https://mobile.twitter.com/chadloder/status/1267011092045115392'

In [ ]:
df_2020pb.to_json()

'{"src":{"0":"https:\\/\\/www.youtube.com\\/watch?v=s7MM1VauRHo","1":"https:\\/\\/mobile.twitter.com\\/chadloder\\/status\\/1267011092045115392","2":"https:\\/\\/twitter.com\\/gunduzbaba1905\\/status\\/1266937500607614982","3":"https:\\/\\/www.reddit.com\\/r\\/Bad_Cop_No_Donut\\/comments\\/gtt3w8\\/i_caught_the_moment_seattle_police_pepper_sprayed\\/","4":"https:\\/\\/www.fox10phoenix.com\\/news\\/video-shows-milk-poured-over-face-of-child-pepper-sprayed-in-seattle-protest","5":"https:\\/\\/www.reddit.com\\/r\\/Seattle\\/comments\\/gu3qq1\\/cop_just_casually_tosses_tear_gas_at_my_feet_like\\/","6":"https:\\/\\/twitter.com\\/EDDIFUL\\/status\\/1267338642617364481","7":"https:\\/\\/twitter.com\\/The_Stepover\\/status\\/1267236742278463488","8":"https:\\/\\/twitter.com\\/The_Stepover\\/status\\/1267237940184231936","9":"https:\\/\\/www.reddit.com\\/r\\/Seattle\\/comments\\/gv0ru3\\/this_is_the_moment_it_all_happened\\/","10":"https:\\/\\/twitter.com\\/Acyn\\/status\\/1267673936659021830?s

In [ ]:
# Generate geolocation from city/state information
def get_geo(df,city_state_df):
    for i in range(len(df)):
        for j in range(len(city_state_df)):
            if df['state'][i] in city_state_df['admin_name'][j] and df['city'][i] in city_state_df['city_ascii'][j]:
                df['lat'][i] = city_state_df['lat'][j]
                df['long'][i] = city_state_df['lng'][j]
        # else: 
        #     return ""

get_geo(df_2020pb, city_state_df)

df_2020pb.head()

,src,state,city,desc,tags,geolocation,title,date,date_text,id,lat,long
0,https://www.youtube.com/watch?v=s7MM1VauRHo,Washington,Olympia,Footage shows a few individuals break off from...,"[arrest, less-lethal, projectile, protester, s...",,Police respond to broken windows with excessiv...,2020-05-31,May 31st,wa-olympia-1,47.0417,-122.896
1,https://mobile.twitter.com/chadloder/status/12...,Washington,Seattle,Officer pins protester with his knee on his ne...,"[arrest, knee-on-neck, protester]",,Officer pins protester by pushing his knee int...,2020-05-30,May 30th,wa-seattle-1,47.6211,-122.324
2,https://twitter.com/gunduzbaba1905/status/1266...,Washington,Seattle,A couple of police officers are seen beating a...,"[beat, protester, punch]",,Police beat unarmed man on the ground,2020-05-31,May 31st,wa-seattle-2,47.6211,-122.324
3,https://www.reddit.com/r/Bad_Cop_No_Donut/comm...,Washington,Seattle,A police officer randomly and indiscriminately...,"[less-lethal, pepper-spray, protester]",,Police indiscriminately pepper spray peaceful ...,2020-05-31,May 31st,wa-seattle-3,47.6211,-122.324
4,https://www.fox10phoenix.com/news/video-shows-...,Washington,Seattle,Police pepper sprays a young child who is seen...,"[child, inhumane-treatment, less-lethal, peppe...",,Police pepper spray young child,2020-05-31,May 31st,wa-seattle-4,47.6211,-122.324


In [ ]:
import json

result = df_2020pb.to_json(orient="records")
parsed = json.loads(result)
json.dumps(parsed)  

'[{"src": "https://www.youtube.com/watch?v=s7MM1VauRHo", "state": "Washington", "city": "Olympia", "desc": "Footage shows a few individuals break off from a protest to smash City Hall windows. Protesters shout at vandals to stop.\\n\\nPolice then arrive. They arrest multiple individuals near the City Hall windows, including one individual who appeared to approach the vandals in an effort to defuse the situation.\\n\\nPolice fire tear gas and riot rounds at protesters during the arrests. Protesters become agitated.\\n\\nAfter police walk arrestee away, protesters continue to shout at police. Police respond with a second bout of tear gas and riot rounds.\\n\\nA racial slur can be heard shouted, although it is unsure who is shouting.", "tags": ["arrest", "less-lethal", "projectile", "protester", "shoot", "tear-gas"], "geolocation": "", "title": "Police respond to broken windows with excessive force", "date": "2020-05-31", "date_text": "May 31st", "id": "wa-olympia-1", "lat": 47.0417, "lon